In [9]:
from langchain.output_parsers.openai_tools import JsonOutputToolsParser
from langchain_community.chat_models import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableLambda
from langchain.chains import create_extraction_chain
from typing import Optional, List
from langchain.chains import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain import hub
import os

os.environ["OPENAI_API_BASE"] = "http://localhost:11434/v1"
#os.environ["OPENAI_MODEL_NAME"] = "gpt-4-1106-preview"
os.environ["OPENAI_API_KEY"] = "NA"

In [10]:
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import PydanticOutputParser
import re



llm = ChatOpenAI(model='llama3')
prompt = hub.pull("ritwik/propositional-indexing")

'''class Proposition(BaseModel):
    "Returns a list of sentences from json-esque string."
    sentence: List[str] = Field(description="Append sentences within double quotations into a list")

#class Data(BaseModel):
Sentences: List[Proposition]'''

#parser = PydanticOutputParser(pydantic_object=Proposition)
#print(parser.get_format_instructions())

runnable = prompt | llm   #runnable to combine the chain -> returns  str(json-esque structure)

#Converts LLM generated json-esque string of propositions -> List of strings(propositions)
def extract_propositions(text) -> List:

    pattern = re.compile(r'\[(.*?)\]', re.DOTALL)
    match = pattern.search(text)
    content = match.group(1)
    propositions = []

    for itm in content.split(','):
        sentence =itm.strip().strip('"') 
        propositions.append(sentence)
        
    return propositions




In [16]:
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader


node_parser = SentenceSplitter(chunk_size=256, chunk_overlap=0)

doc_pages = SimpleDirectoryReader(input_dir="/home/ritwik-gosh/Fine_tuning/data/", exclude=["ACA.pdf"]).load_data()
#print(doc[1].page_content)


chunks = node_parser.get_nodes_from_documents(doc_pages)
'''for idx in range(len(doc_pages)):
    page_content = doc_pages[idx].text
    rec_chunks.extend(node_parser.get_nodes_from_documents([page_content]))'''

    #single_sentences_list.append(re.split(r'(?<=[.?!])\s+', page_content))

all_props = []
for idx in range(1):
    chunk = chunks[idx].text
    runnable_output = runnable.invoke({"input": chunk}).content
    all_props.extend(extract_propositions(runnable_output))



In [22]:
print(all_props)

    




['Nedyalko Peshev', 'Valentin Tsenev investigated the effect of solder material quantity on obtaining strong solder joints in LGA type assembly.', 'The researchers analyzed results of their research on the strength of solders of LGA (Land Grid Array) assembly type at different amounts of soldering material and using Hot bar soldering technology.', 'Experiments were described for achieving the maximum tensile strength of soldered pads on a flexible circuit board (FPC) to a standard rigid circuit board (PCB) at different amounts of solder by Hot bar soldering.', 'The method of measuring the amount of solder material was specified and the setup for measuring solder height was described.', 'The soldering system of flexible and rigid board was applied', 'and the studied pads were described.']


In [70]:
paragh = "Harper was the daughter of Mr. Bane who she never recognised until her first birthday.It is astonishingly weird."

for chunk in runnable.stream({"input": paragh}):
    print(chunk.content, end="", flush=True)

Here is the decomposed list of propositions:

["Harper's parent is a person called Mr. Bane.", 
"Mr. Bane is Harper's father.",
"Harper did not recognize Mr. Bane as her parent until her first birthday.", 
"It is surprising that this is the case.", 
"The situation is unusual."]

In [12]:
# Pydantic data class
from langchain.output_parsers import PydanticOutputParser

class Sentences(BaseModel):
    '''
    This class is used to extract the sentences from the output of an LLM and store them one-by-one into a list.
    '''
    sentences: List[str]

output_parser = PydanticOutputParser(pydantic_object=Sentences)
format_instructions = output_parser.get_format_instructions()

    
'''# Extraction
extraction_chain = create_extraction_chain_pydantic(pydantic_schema=Sentences, llm=llm)
propositions = extraction_chain.run(out).sentences'''


'# Extraction\nextraction_chain = create_extraction_chain_pydantic(pydantic_schema=Sentences, llm=llm)\npropositions = extraction_chain.run(out).sentences'

In [13]:
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"description": "This class is used to extract the sentences from the output of an LLM and store them one-by-one into a list.", "properties": {"sentences": {"title": "Sentences", "type": "array", "items": {"type": "string"}}}, "required": ["sentences"]}
```
